# Fine-tuning a Model for Summarization Task

In this task, you will load, preprocess, and fine-tune a T5 model on a dataset of news articles for a summarization task. Follow the steps below carefully.

### Model and Dataset Information

For this task, you will be working with the following:

- **Model Checkpoint**: Use the pre-trained model checkpoint `UBC-NLP/AraT5-base` if you face any problem you can use `google-t5/t5-small` but the first one is the correct one for both the model and tokenizer.
- **Dataset**: You will be using the `CUTD/news_articles_df` dataset. Ensure to load and preprocess the dataset correctly for training and evaluation.

**Note:**
- Any additional steps or methods you include that improve or enhance the results will be rewarded with bonus points if they are justified.
- The steps outlined here are suggestions. You are free to implement alternative methods or approaches to achieve the task, as long as you explain the reasoning and the process at the bottom of the notebook.
- You can use either TensorFlow or PyTorch for this task. If you prefer TensorFlow, feel free to use it when working with Hugging Face Transformers.
- The number of data samples you choose to work with is flexible. However, if you select a very low number of samples and the training time is too short, this could affect the evaluation of your work.

## Step 1: Load the Dataset

In [1]:
!pip install datasets # corrected module name

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires 

In [2]:
!pip install --upgrade accelerate

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [3]:
!pip install transformers datasets sklearn

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [4]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration

Load the dataset and split it into training and test sets. Use 20% of the data for testing.

In [13]:

dataset = load_dataset("CUTD/news_articles_df")
split_dataset = dataset['train'].train_test_split(test_size=0.2, seed=42)
train_dataset = split_dataset['train']
test_dataset = split_dataset['test']


In [14]:
train_dataset[:3]

{'Unnamed: 0': [8065, 4521, 6235],
 'summarizer': ['\nأعلنت التنسيقية الوطنية لشبكة دستورنا في بيان أصدرته اليوم الاثنين انسحابها من عضوية جبهة الانقاذ الوطني. وأعتبرت أن مهمتها التي قامت بها كجمعية مواطنية داخل هذه الجبهة لم يعد لها مبرر،وفق نص البيان.',
  'و أوضح إقليم الشركة \xa0في بلاغ له اليوم أن قطع التيار الكهربائي يأتي بغاية انجاز أشغال صيانة بالشبكة .',
  'يشار الى ان اغلب المنتصبين بالمكان تفاعلوا ايجابا مع هذه الحملة من خلال الرفع الارادي للمظلات وغيرها بالمكان. وأضاف ان هذه الحملة الرقابية الاستباقية تندرج في اطار الاستراتيجية الجهوية للقضاء على كل مظاهر الاعتداء على النصوص القانونية.'],
 'text': ['اعلنت التنسيقيه الوطنيه لشبكه دستورنا بيان اصدرته اليوم الاثنين انسحابها عضويه جبهه الانقاذ الوطني واعتبرت مهمتها قامت كجمعيه مواطنيه داخل الجبهه يعد مبرر وفق نص البيان',
  'اعلن اقليم الشركه التونسيه للكهرباء والغاز بالقصرين انه سيتم قطع التيار الكهربائي كامل معتمديه حاسي الفريد مناطق المزيرعه نوبه الكامور القويره يوم غد الاحد ابتداء الساعه السابعه صباحا الى غايه الساعه الواحده 

## Step 2: Load the Pretrained Tokenizer

Initialize a tokenizer from the gevin model checkpoint.

In [15]:
tokenizer = AutoTokenizer.from_pretrained("UBC-NLP/AraT5-base")

tokenizer_config.json:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/2.44M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


## Step 3: Preprocess the Dataset

Define a preprocessing function that adds a prefix ("summarize:") to each input if needed and tokenizes the text for the model. The labels will be the tokenized summaries.

In [22]:
def preprocess_function(examples):

    inputs = ["summarize: " + doc for doc in examples["text"]]

    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summarizer"], max_length=150, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/6702 [00:00<?, ? examples/s]

Map:   0%|          | 0/1676 [00:00<?, ? examples/s]

## Step 4: Define the Data Collator

Use a data collator designed for sequence-to-sequence models, which dynamically pads inputs and labels.

## Step 5: Load the Pretrained Model

Load the model for sequence-to-sequence tasks (summarization).

In [23]:


model = T5ForConditionalGeneration.from_pretrained("UBC-NLP/AraT5-base")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)


## Step 6: Define Training Arguments

Set up the training configuration with parameters like learning rate, batch size, and number of epochs.

In [24]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
)


## Step 7: Initialize the Trainer

Use the `Seq2SeqTrainer` class to train the model.

In [25]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


## Step 8: Fine-tune the Model

Train the model using the specified arguments and dataset.

In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss


## Step 9: Inference

Once the model is trained, perform inference on a sample text to generate a summary. Use the tokenizer to process the text, and then feed it into the model to get the generated summary.

In [ ]:
sample_text = '''أعلنت القيادة العامة لشرطة دبي، يوم السبت، أن فهد المولد لاعب المنتخب السعودي سقط من شرفة منزله بالدور الثاني.

ونشرت شرطة دبي بيانا يوم السبت عن الحادث جاء فيه: أظهرت التحقيقات الأولية أن الحادث وقع أثناء وجوده في الشرفة.

وأضافت: الحالة الصحية للاعب فهد المولد تحت المراقبة حيث يرقد في العناية المركزة في أحد مستشفيات دبي، حيث تتواصل جهود الفرق الطبية لتقديم الرعاية اللازمة له. وفي إطار التزامها بالشفافية، تؤكد القيادة العامة لشرطة دبي أن التحقيقات مستمرة، وسيتم إبلاغ الجهات المختصة بنتائج هذه التحقيقات حال الانتهاء منها.

وأصدرت القنصلية العامة السعودية بيانا تؤكد فيه متابعتها لحادث فهد المولد والتحقيقات المتعلقة به.
'''
inputs = tokenizer("summarize: " + sample_text, return_tensors="pt", max_length=512, truncation=True)
summary_ids = model.generate(inputs["input_ids"])

print(tokenizer.decode(summary_ids[0], skip_special_tokens=True))
